# AA/EE/ME 548: Homework 03

## Name: Yenpang Huang

### Export page instructions:
Export the PDF as though you were going to print the page in your browser (Ctrl+P) rather than through Colab's print functionality, otherwise some of the text will be cut off.

### Homework feedback:

**What was the coolest thing(s) you learned from lectures/homework that you didn't know previously?**

The application of LQR

**Which problem did you find the most challenging and why?**

2

**Approximate how long did you spend on this homework?**
5 days

**Feedback on this homework (can improve for future hw and years)**




## Submission details
To submit your homework, please save a PDF copy of your notebook (make sure all the cells are run correctly, and the correct outputs are displayed), and submit via Canvas.

In [1]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

## Problem 1

### (a)
Assume non-time0varying case Q, R, S, A, B

Following the LQR derivation in class, we obtain
$$
\min_{u}\left[u_{N-1}^T R u_{N-1} + 2x_{N-1}^T S u_{N-1} + 2x_{N-1}^T A^T P_N B u_{N-1} + u_{N-1}^T B^T P_N B u_{N-1} \right]
$$

then,

$$
2R u_{N-1} + 2x_{N-1}^T S + 2B^T P_N A x_{N-1} + 2B^T P_N B u_{N-1} = 0
$$

$$
2(R + B^T P_N B) u_{N-1} = -2(S + B^T P_N A) x_{N-1}
$$

$$
u_{N-1}^* = \left[ - \left( R +  B^T P_N B \right)^{-1} (S + B^T P_N A) \right]x_{N-1}
$$

hence,

$$
K_k = \left[ - \left( R +  B^T P_{k+1} B \right)^{-1} (S + B^T P_{k+1} A) \right]
$$

$$
P_k = Q + A^T P_{k+1} A - A^T P_{k+1} B \left( R +  B^T P_{k+1} B \right)^{-1} (S + B^T P_{k+1} A)
$$



# Problem 2

The code below was the part that has been modified, and a screenshot of the output animation

In [2]:
#####################

def simulate_closed_loop(initial_state, nominal_states, nominal_controls):
    states = [initial_state]
    Klist = []
    PN = Q

    for k in range(N):
        ### UPDATE ME! (AND ADD CODE ELSEWHERE AS YOU SEE FIT) ###
        control = nominal_controls[k]
        x_error = states[k] - nominal_states[k]
        A, B = jax.jacobian(planar_quad.discrete_step, [0, 1])(nominal_states[-1-k], nominal_controls[-1-k], dt)

        P = (Q + A.T @ PN @ A - (A.T @ PN @ B) @
                jnp.linalg.inv(R + B.T @ PN @ B) @ (B.T @ PN @ A))
        K = -jnp.linalg.inv(R + B.T @ PN @ B) @ (B.T @ PN @ A)
        PN = P
        Klist.append(K)
        control += Klist[k] @ x_error

        ##########################################################

        control = np.clip(control, planar_quad.min_thrust_per_prop, planar_quad.max_thrust_per_prop)
        next_state = planar_quad.discrete_step(states[k], control, dt)
        next_state = apply_wind_disturbance(next_state, dt)
        states.append(next_state)

    return np.array(states)


# planar_quad = PlanarQuadrotor()
# fig, ax = plt.subplots(figsize=(12, 6))
# plot_obstacle(ax)
# plot_nominal_trajectory(ax)
# plot_wind(ax)
# planar_quad.animate(simulate_closed_loop(initial_state, nominal_states, nominal_controls), dt, ax)

In [3]:
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt

drive.mount('/content/drive')

image_path = '/content/drive/MyDrive/ME 548/HW 3/TLQR.png'

image = Image.open(image_path)

image.show()

MessageError: Error: credential propagation was unsuccessful

## 2(b) Why does there exist some "steady state error" at the end of the trajectory?

The "steady state error" at the end may be due to control limits.

## 2(c) If we found ourselves running up against control limits (i.e., activating the `np.clip` in the cell above; this shouldn't be the case with the numbers given in this problem as written), what could we change in (a) the tracking LQR formulation, or (b) the computation of the nominal trajectory, to make this less likely to happen?

We could address control limits issue in two way. First, we could increase the values of R matrix in the cost function of tracking LQR to set higher control penalties. Secondly, we could also add additional control constraints in the optimize_nominal_trajectory function to overcome this issue.


## 2(d) Even with closed-loop control, we see that the red "safety bubble" surrounding the quad intersects the obstacle over a short time interval. What could we do to avoid this?

We could add constraints to the 'safety bubble' of the drone to avoid collision with the obstacle. Another approach would be to implement a CBF or perform HJ Reachability to achieve the same goal.

# Problem 3

In [ ]:
!pip install --upgrade hj-reachability

In [ ]:
from IPython.display import HTML
import matplotlib.animation as anim
import plotly.graph_objects as go

import hj_reachability as hj
from hj_reachability import dynamics
from hj_reachability import sets

In [ ]:

class DynUnicycle(dynamics.ControlAndDisturbanceAffineDynamics):

    def __init__(self,
                 max_turn_rate=1.,
                 max_accel=2.,
                 control_mode="max",
                 disturbance_mode="min",
                 control_space=None,
                 disturbance_space=None,
                 alpha=0.):
        # control and disturbance bounds
        # in this set up, the disturbance bounds are proportional to the control
        # bounds by a factor of alpha
        if control_space is None:
            control_space = sets.Box(lo=jnp.array([-max_turn_rate, -max_accel]), hi=jnp.array([max_turn_rate, max_accel]))
        if disturbance_space is None:
            disturbance_space = sets.Box(lo=jnp.array([-max_turn_rate * alpha, -max_accel * alpha]), hi=jnp.array([max_turn_rate * alpha, max_accel * alpha]))
        super().__init__(control_mode, disturbance_mode, control_space, disturbance_space)

    def open_loop_dynamics(self, state, time):
        ### PUT YOUR CODE HERE ###

        _, _, theta, v = state

        return jnp.array([v * jnp.cos(theta), v * jnp.sin(theta), 0, 0])

    def control_jacobian(self, state, time):
        ### PUT YOUR CODE HERE ###

        x, y, _, _ = state
        return jnp.array([
            [0., 0.],
            [0., 0.],
            [1., 0.],
            [0., 1.]
        ])

    def disturbance_jacobian(self, state, time):
        ### PUT YOUR CODE HERE ###

        return jnp.array([
            [0., 0.],
            [0., 0.],
            [1., 0.],
            [0., 1.]
        ])


In [ ]:

### REACH ###
# control_mode = "min"
# disturbance_mode = "max"
####################################

### AVOID or when computing FRS ###
control_mode = "max"
disturbance_mode = "min"
####################################

# model with noise/disturbance bounds proportional to the control limits
dynamics_noisy = DynUnicycle(alpha=0.6, control_mode=control_mode, disturbance_mode=disturbance_mode)

# no noise on the model
dynamics = DynUnicycle(control_mode=control_mode, disturbance_mode=disturbance_mode)

grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(lo=np.array([-10., -10., 0., -4.]),
                                                                   hi=np.array([10., 10., 2 * np.pi, 4.])), (21, 21, 30, 31),
                                                       periodic_dims=2)
# terminal values. circle in position space centered at origin and with radius 3
values = jnp.linalg.norm(grid.states[..., :2], axis=-1) - 3
solver_settings = hj.SolverSettings.with_accuracy("very_high",
                                                  hamiltonian_postprocessor=hj.solver.backwards_reachable_tube)

In [ ]:
# solve the HJI equation
time = 0.
target_time = -2
target_values = hj.step(solver_settings, dynamics, grid, time, values, target_time).block_until_ready()
target_values_noisy = hj.step(solver_settings, dynamics_noisy, grid, time, values, target_time).block_until_ready()


In [ ]:
## plotting
fontsize = 16
## When plotting, use the following indices for theta and v when
## slicing the value function across the x and y dimensions.
theta_i = 0
v_i = 25

plt.jet()
plt.figure(figsize=(8, 6))
plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], target_values[:, :, theta_i, v_i].T)
cbar = plt.colorbar()
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            values[:, :, theta_i, v_i].T,
            levels=0,
            colors="black",
            linewidths=3)
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            target_values[:, :, theta_i, v_i].T,
            levels=0,
            colors="red",
            linewidths=3)
plt.contour(grid.coordinate_vectors[0],
            grid.coordinate_vectors[1],
            target_values_noisy[:, :, theta_i, v_i].T,
            levels=0,
            colors="yellow",
            linewidths=3)



plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
ticklabs = cbar.ax.get_yticklabels()
cbar.ax.set_yticklabels(ticklabs, fontsize=fontsize)
plt.axis("equal")
plt.tight_layout()

# Problem 4

## 4(a)

Based on Euler integration
$$ x_{t+1} = x_t + \Delta t f(x_t, u_t, t) $$
$$ P_{x_{t+1}} = P_{x_t} + \Delta t \cdot v_{x_t} $$
$$ P_{y_{t+1}} = P_{y_t} + \Delta t \cdot v_{y_t} $$
$$ v_{x_{t+1}} = v_{x_t} + \Delta t \cdot a_{x_t} $$
$$ v_{y_{t+1}} = v_{y_t} + \Delta t \cdot a_{y_t} $$

## 4(b)

$$ \min_{x, u} = \sum_{k=0}^{T-1} \|u^2\| $$
$$
\begin{matrix}
 s.t. & u_k = U, k = 0, ..., N-1 \\
& x_k = X, x_k \notin 0, k=0, ..., N
\end{matrix}
$$
\
\
$$ x_{k+1} = f(x_k, u_k, k), k=0, ..., N-1 $$
$$ x_0 = x_{start} $$
$$ x_N = x_{final} $$
$$ \| u \|_2 \leq a_{max} $$

In [ ]:
import cvxpy as cp

T = 50  # number of time steps
n = 4   # state dimension
m = 2   # control dimension

xs = cp.Variable([T+1,n]) # state trajectory variable
us = cp.Variable([T,m])   # control trajectory variable

### Add your code here
x_start = np.array([-4, 0, -1, 2]).T
x_final = np.array([0, 0, 2, 0]).T
dt = 0.1
amax = 2

A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

B = np.array([[0, 0],
             [0, 0],
             [dt, 0],
             [0, dt]])

cost = 0
constr = []
for t in range(T):
    cost += cp.sum_squares(us[t, :])
    constr += [xs[t + 1, :] == A @ xs[t, :] + B @ us[t, :], cp.norm(us[t, :]) <= amax]

constr += [xs[T, :] == x_final, xs[0, :] == x_start]
problem = cp.Problem(cp.Minimize(cost), constr)
problem.solve()

######################


# plotting out results.
states = xs.value
controls = us.value

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(states[:,0], states[:,1])
plt.quiver(states[:,0], states[:,1], states[:,2], states[:,3], scale=30, width=.003)
plt.quiver(states[:-1,0], states[:-1,1], controls[:,0], controls[:,1], scale=30, width=.003, color='red')
plt.title("Optimal Trajectory with velocity and acceleration vector")
plt.xlabel("$x$ position")
plt.ylabel("$y$ position")
plt.grid()

plt.subplot(1,2,2)
plt.plot(controls)
plt.plot(np.linalg.norm(controls, 2, axis=1), '--')
plt.title("Optimal Control Sequence")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration")
plt.legend(["$a_x$", "$a_y$", "$\|a\|_2$"])
plt.grid()